In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
from sklearn.metrics import f1_score, precision_recall_curve

In [44]:
subsets = ['val','test']
threshold = 0.5

for subset in subsets: 
    result_file = 'Embedding-{}-results.txt'.format(subset)
    fp = open(result_file)
    lines = fp.readlines() 
    lines = [x.strip() for x in lines]
    fp.close()
    print(lines[:10])
    
    y_true = []
    y_pred = [] 
    y_prob = [] 

    covid_ids = [] 
    non_covid_ids = []     
    for l in lines: 
        l_splits = l.split(',') 
        #print(l_splits)    
        #change covid class label to 1 
        truth = int(l_splits[1])
        #print(truth)
        y_true.append(truth)
        
        num_preds = int((len(l_splits)-2)/3)
        prob0s = [] 
        preds = [] 
        for i in range(num_preds):        
            pred = int(l_splits[2+3*i])        
            s0 = float(l_splits[2+3*i+1])
            s1 = float(l_splits[2+3*i+2])        
            prob0 = np.exp(s0)/(np.exp(s0)+np.exp(s1)) 
            preds.append(pred)
            prob0s.append(prob0)
        
        prob0s = np.asarray(prob0s)    
        prob0s_avg = np.mean(prob0s)    
        pred2 = (prob0s_avg < threshold).astype(np.uint8) 
        #print(true, pred2, preds, prob0s)
        y_pred.append(pred2) 
        y_prob.append(prob0s_avg)   
        #input('dbg')
        if pred2==0:
            covid_ids.append(l_splits[0])
        else: 
            non_covid_ids.append(l_splits[0])
    
    y_pred = np.asarray(y_pred)
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
        
    print(len(y_true),len(y_pred))
        
    F1_1 = f1_score(y_true, y_pred, average='micro')
    F1_2 = f1_score(y_true, y_pred, average='macro')
    print(F1_1,F1_2)
    
    fp = open('Embedding-{}-{}-covid.csv'.format(threshold,subset),'w')    
    len_covid = len(covid_ids) 
    for i in range(len_covid): 
        
        if subset=="test":
            ids = covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]        
        else:
            ids = covid_ids[i][4:]        
        #print(covid_ids[i],ids)   
            
        if i<len_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 

    fp = open('Embedding-{}-{}-non-covid.csv'.format(threshold,subset),'w')
    len_non_covid = len(non_covid_ids) 
    for i in range(len_non_covid):
        if subset=="test":
            ids = non_covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]        
        else:
            ids = non_covid_ids[i][4:]

        if i<len_non_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 
    

['val_covid_ct_scan_0,0,0,2.498,-1.985,0,2.507,-1.994,0,2.518,-2.020,0,2.545,-2.029,0,2.459,-2.133', 'val_covid_ct_scan_1,0,0,2.485,-2.129,0,2.422,-2.062,0,2.501,-2.130,0,2.522,-2.174,0,2.011,-1.827', 'val_covid_ct_scan_2,0,0,1.184,-1.170', 'val_covid_ct_scan_3,0,0,2.243,-1.944', 'val_covid_ct_scan_4,0,0,2.168,-1.934,0,2.060,-1.838', 'val_covid_ct_scan_5,0,0,1.991,-1.824,0,1.958,-1.843,0,2.140,-1.946,0,1.935,-1.823,0,0.619,-0.570', 'val_covid_ct_scan_6,0,0,2.793,-2.319,0,2.803,-2.306,0,2.830,-2.345,0,2.777,-2.292,0,2.751,-2.275,0,2.808,-2.372', 'val_covid_ct_scan_7,0,0,2.886,-2.336', 'val_covid_ct_scan_8,0,0,2.409,-2.249', 'val_covid_ct_scan_9,0,0,1.849,-1.707,0,1.951,-1.689,0,1.864,-1.610,0,1.933,-1.656,1,-0.131,0.641']
374 374
0.9064171122994652 0.9049154844661393
['test_subset1_3ea85c05-2cd1-4b5a-822f-3985b0e06867,0,1,-0.489,0.341,0,0.191,-0.353,1,-0.364,0.436,1,-0.853,0.927,1,-2.605,2.107', 'test_subset1_2c442cd2-0abd-4ebd-9223-ff7ee8e9953c,0,1,-4.329,4.130,1,-4.329,4.130', 'test_s